In [5]:
import glob, pathlib, json

In [ ]:
fns = glob.glob(r"D:\ig_pipeline\cad\*\*")
has_tree = []
empty = []
for fn in fns:
    p = pathlib.Path(fn) / "artifacts" / "object_list.json"
    if not p.exists():
        continue
    with open(p, "r") as f:
        d = json.load(f)
        if len(d["needed_objects"]) == 0:
            empty.append(fn)
        if "meta_links" not in d:
            continue
        if any("collision" in v for v in d["meta_links"].values()):
            has_tree.append(fn)

In [ ]:
print("dvc unprotect", " ".join(str(pathlib.Path(x).relative_to(pathlib.Path(r"D:\ig_pipeline"))) for x in empty))

In [ ]:
print("\n".join("rm -Recurse " + str(pathlib.Path(x).relative_to(pathlib.Path(r"D:\ig_pipeline"))) for x in empty))

In [ ]:
len(has_tree)

In [ ]:
has_tree[0]

In [ ]:
missing = set(fns) - set(has_tree)

In [ ]:
len(missing)

In [ ]:
target_fns = []
for m in sorted(missing)[:100]:
    fn = pathlib.Path(m) / "artifacts/object_list.json"
    if not fn.exists():
        continue
    relative_fn = fn.relative_to(pathlib.Path(r"D:\ig_pipeline"))
    target_fns.append(str(relative_fn))

print("dvc unprotect", " ".join(target_fns))

In [ ]:
for m in sorted(missing)[:100]:
    fn = pathlib.Path(m) / "processed.max"
    relative_fn = fn.relative_to(pathlib.Path(r"D:\ig_pipeline"))
    print(f"python b1k_pipeline/batch_3dsmax.py {relative_fn} b1k_pipeline/max/object_list.py")

In [ ]:
to_commit = []
for fn in fns:
    p = pathlib.Path(fn) / "artifacts" / "object_list.json"
    if p.exists() and not p.is_symlink():
        relative_fn = p.relative_to(pathlib.Path(r"D:\ig_pipeline"))
        to_commit.append(str(relative_fn).replace("\\", "/"))
print("dvc commit", " ".join(to_commit[:50]))

In [15]:
# Get all the object targets, split into two, and run a bunch of them
import sys
sys.path.append(r"D:\ig_pipeline")
from b1k_pipeline.utils import get_targets
t = get_targets("objects")

import random
random.seed(1234)
random.shuffle(t)
assert "objects/compost-pf" not in t

In [3]:
i = 1
side = t[i::2]
for start in range(0, len(side), 100):
    batch = side[start:start+100]
    print("dvc repro", " ".join(f"export_meshes@{x}" for x in batch))
print("")

dvc repro export_meshes@objects/legacy_folder-poqpux export_meshes@objects/legacy_armchair-fsrprw export_meshes@objects/legacy_almond-acooui export_meshes@objects/oyster-xp export_meshes@objects/legacy_perfume-ipurzb export_meshes@objects/beer_keg-rn export_meshes@objects/pizza_box-vm export_meshes@objects/legacy_washer-zgzvcv export_meshes@objects/allspice-vw export_meshes@objects/clamp-ok export_meshes@objects/deicer-sh export_meshes@objects/legacy_bucket-bdhvnt export_meshes@objects/caulk-hi export_meshes@objects/legacy_pencil-vakowb export_meshes@objects/legacy_toilet-chuack export_meshes@objects/aspirin-sr export_meshes@objects/legacy_bottom_cabinet-mxrzpj export_meshes@objects/legacy_pot_plant-ftbsdt export_meshes@objects/legacy_lemon-vuwehu export_meshes@objects/coffee_grinder-ag export_meshes@objects/ammonia-sd export_meshes@objects/cd-ui export_meshes@objects/legacy_folding_chair-dqhazh export_meshes@objects/legacy_caliper-sngjmj export_meshes@objects/baseball_glove-cs export_

In [14]:
len(glob.glob(r"D:\ig_pipeline\cad\objects\*\artifacts\meshes.zip"))

742

In [19]:
sum(1 for x in t if (pathlib.Path(r"D:\ig_pipeline\cad") / x / "artifacts/meshes.zip").exists())

743